<a href="https://colab.research.google.com/github/Krishnaugale353/Pnemonia-Detection-using-ML/blob/main/Pnemonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tensorflow-gpu


In [2]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob


In [3]:
IMAGESHAPE = [224, 224, 3]
training_data = '/content/drive/MyDrive/chest_xray/chest_xray/train'
testing_data = "/content/drive/MyDrive/chest_xray/chest_xray/test"


In [4]:
!nvidia-smi

Sun Feb 11 11:25:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)


In [6]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [7]:
classes = glob('/content/drive/MyDrive/chest_xray/chest_xray/test/*')


In [8]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [9]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [11]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [12]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/chest_xray/train',
												target_size = (224, 224),
												batch_size = 6,
												class_mode = 'categorical')


Found 5216 images belonging to 2 classes.


In [13]:
test_set = testing_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/chest_xray/test',
											target_size = (224, 224),
											batch_size = 6,
											class_mode = 'categorical')


Found 640 images belonging to 2 classes.


In [14]:
print("Size of the training dataset:", len(training_set))
print("Size of the testing dataset:", len(test_set))

Size of the training dataset: 870
Size of the testing dataset: 107


In [15]:
fitted_model = final_model.fit_generator(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)


<ipython-input-15-0133b02f9370>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


Epoch 1/5
870/870 [==============================] - 670s 765ms/step - loss: 0.2563 - accuracy: 0.9235 - val_loss: 2.1167 - val_accuracy: 0.6750
Epoch 2/5
870/870 [==============================] - 139s 160ms/step - loss: 0.2290 - accuracy: 0.9442 - val_loss: 0.8794 - val_accuracy: 0.8734
Epoch 3/5
870/870 [==============================] - 140s 161ms/step - loss: 0.1506 - accuracy: 0.9597 - val_loss: 0.4288 - val_accuracy: 0.9141
Epoch 4/5
870/870 [==============================] - 137s 157ms/step - loss: 0.1668 - accuracy: 0.9605 - val_loss: 1.0779 - val_accuracy: 0.8438
Epoch 5/5
870/870 [==============================] - 138s 158ms/step - loss: 0.1853 - accuracy: 0.9571 - val_loss: 1.1490 - val_accuracy: 0.8531


In [16]:
final_model.save('our_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [18]:
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('our_model.h5') #Loading our model
img=image.load_img('/content/drive/MyDrive/chest_xray/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


1/1 [==============================] - 1s 1s/step
Person is affected with Pneumonia.
Predictions: [[0. 1.]]


In [19]:
from sklearn.metrics import classification_report, confusion_matrix


# Evaluate the model on the testing set
evaluation = final_model.evaluate(test_set, steps=len(test_set))

# Print overall accuracy
print("Testing Accuracy: {:.2f}%".format(evaluation[1] * 100))

# Make predictions on the test set
predictions = final_model.predict(test_set, steps=len(test_set))

# Convert predictions to binary values (0 or 1)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_set.classes

# Print confusion matrix and classification report
conf_matrix = confusion_matrix(true_classes, predicted_classes)
classification_rep = classification_report(true_classes, predicted_classes, target_names=classes)

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_rep)


107/107 [==============================] - 8s 73ms/step - loss: 1.1490 - accuracy: 0.8531
Testing Accuracy: 85.31%
107/107 [==============================] - 14s 133ms/step

Confusion Matrix:
[[ 53 189]
 [103 295]]

Classification Report:
                                                             precision    recall  f1-score   support

   /content/drive/MyDrive/chest_xray/chest_xray/test/NORMAL       0.34      0.22      0.27       242
/content/drive/MyDrive/chest_xray/chest_xray/test/PNEUMONIA       0.61      0.74      0.67       398

                                                   accuracy                           0.54       640
                                                  macro avg       0.47      0.48      0.47       640
                                               weighted avg       0.51      0.54      0.52       640



In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# Data generators for training and testing
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Load the training dataset
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/chest_xray/chest_xray/train',
    target_size=(224, 224),
    batch_size=6,
    class_mode='categorical'
)

# Load the test dataset
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/chest_xray/chest_xray/test',
    target_size=(224, 224),
    batch_size=6,
    class_mode='categorical'
)

# Initialize and train the SVM model
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=2))

# Extract images and labels from the entire training set
train_images = []
train_labels = []

for images_batch, labels_batch in training_set:
    train_images.append(images_batch)
    train_labels.append(labels_batch)
    if len(train_images) * training_set.batch_size >= len(training_set.filenames):
        break

# Concatenate the batches into a single array
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)

# Reshape the data
train_images_flattened = train_images.reshape(train_images.shape[0], -1)

# Train the SVM model on the entire training dataset
svm_model.fit(train_images_flattened, np.argmax(train_labels, axis=1))

# Load and accumulate predictions over all batches of the test dataset
all_predictions_svm = []
all_true_labels = []

for images_batch, labels_batch in test_set:
    # Reshape the batch data
    images_batch_flattened = images_batch.reshape(images_batch.shape[0], -1)

    # Predict labels using the SVM model
    predictions_batch = svm_model.predict(images_batch_flattened)

    # Append predictions and true labels
    all_predictions_svm.extend(predictions_batch)
    all_true_labels.extend(np.argmax(labels_batch, axis=1))

# Convert to numpy arrays for easier manipulation
all_predictions_svm = np.array(all_predictions_svm)
all_true_labels = np.array(all_true_labels)

# Print confusion matrix and classification report for SVM
conf_matrix_svm = confusion_matrix(all_true_labels, all_predictions_svm)
classification_rep_svm = classification_report(all_true_labels, all_predictions_svm)

print("\nSVM - Confusion Matrix:")
print(conf_matrix_svm)

print("\nSVM - Classification Report:")
print(classification_rep_svm)


Found 5216 images belonging to 2 classes.
Found 640 images belonging to 2 classes.
